In [3]:
import heapq
import time

class PuzzleState:
    def _init_(self, board, parent=None, move=None, depth=0, cost=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.blank_index = board.index(0)

    def _lt_(self, other):
        return self.cost < other.cost  # For priority queue comparison

    def possible_moves(self):
        moves = []
        row, col = divmod(self.blank_index, 3)
        directions = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1)}
        
        for move, (dr, dc) in directions.items():
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_blank_index = new_row * 3 + new_col
                new_board = self.board[:]
                new_board[self.blank_index], new_board[new_blank_index] = new_board[new_blank_index], new_board[self.blank_index]
                moves.append(PuzzleState(new_board, self, move, self.depth + 1))
        return moves

def manhattan_distance(state):
    goal_positions = {n: (i // 3, i % 3) for i, n in enumerate([1, 2, 3, 4, 5, 6, 7, 8, 0])}
    return sum(abs(r - goal_positions[n][0]) + abs(c - goal_positions[n][1]) for i, n in enumerate(state.board) if n != 0 for r, c in [divmod(i, 3)])

def misplaced_tiles(state):
    return sum(1 for i, n in enumerate(state.board) if n != 0 and n != i + 1)

def solve_puzzle(initial_board, heuristic, algorithm):
    start_time = time.time()
    start_state = PuzzleState(initial_board)
    priority_queue = []
    heapq.heappush(priority_queue, (heuristic(start_state), start_state))

    visited = set()
    nodes_expanded = 0

    while priority_queue:
        _, current_state = heapq.heappop(priority_queue)
        nodes_expanded += 1

        if current_state.board == [1, 2, 3, 4, 5, 6, 7, 8, 0]:
            execution_time = time.time() - start_time
            return reconstruct_path(current_state), nodes_expanded, execution_time

        visited.add(tuple(current_state.board))

        for neighbor in current_state.possible_moves():
            if tuple(neighbor.board) not in visited:
                g_cost = current_state.depth + 1
                h_cost = heuristic(neighbor)
                f_cost = g_cost + h_cost if algorithm == "A*" else h_cost
                
                neighbor.cost = f_cost
                heapq.heappush(priority_queue, (f_cost, neighbor))

    return None, nodes_expanded, time.time() - start_time

def reconstruct_path(state):
    path = []
    while state.parent:
        path.append(state.move)
        state = state.parent
    return path[::-1]

def is_solvable(board):
    flattened = [num for num in board if num != 0]
    inversions = sum(1 for i in range(len(flattened)) for j in range(i + 1, len(flattened)) if flattened[i] > flattened[j])
    return inversions % 2 == 0

def hill_climbing(initial_board, heuristic):
    current_state = PuzzleState(initial_board)
    
    while True:
        neighbors = current_state.possible_moves()
        neighbors.sort(key=heuristic)

        best_neighbor = neighbors[0] if neighbors else None

        if best_neighbor and heuristic(best_neighbor) < heuristic(current_state):
            current_state = best_neighbor
        else:
            break  # Local maximum reached

    return current_state.board

if __name__  == "__main__":
    initial_board = [1, 2, 3, 4, 0, 5, 6, 7, 8]  # Example start state

    if is_solvable(initial_board):
        solution, nodes_expanded, execution_time = solve_puzzle(initial_board, manhattan_distance, "A*")
        print(f"A* Solution: {solution}, Nodes Expanded: {nodes_expanded}, Time: {execution_time:.4f} sec")

        solution, nodes_expanded, execution_time = solve_puzzle(initial_board, misplaced_tiles, "GBFS")
        print(f"GBFS Solution: {solution}, Nodes Expanded: {nodes_expanded}, Time: {execution_time:.4f} sec")

        final_state = hill_climbing(initial_board, manhattan_distance)
        print(f"Hill Climbing Final State: {final_state}")
    else:
        print("The given puzzle configuration is unsolvable.")

TypeError: PuzzleState() takes no arguments

In [5]:
import heapq
import random
import time
from copy import deepcopy

class Puzzle:
    def __init__(self, state, parent=None, move=None, depth=0, cost=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

    def get_blank_position(self):
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == 0:
                    return i, j

    def get_neighbors(self):
        neighbors = []
        x, y = self.get_blank_position()
        moves = [(0, -1, "Left"), (0, 1, "Right"), (-1, 0, "Up"), (1, 0, "Down")]
        for dx, dy, move in moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_state = deepcopy(self.state)
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                neighbors.append(Puzzle(new_state, self, move, self.depth + 1))
        return neighbors

    def get_path(self):
        path, node = [], self
        while node:
            path.append(node.move)
            node = node.parent
        return path[::-1]

def heuristic_misplaced_tiles(state, goal):
    return sum(1 for i in range(3) for j in range(3) if state[i][j] and state[i][j] != goal[i][j])

def heuristic_manhattan(state, goal):
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                x, y = divmod(state[i][j] - 1, 3)
                distance += abs(x - i) + abs(y - j)
    return distance

def a_star(initial, goal, heuristic):
    start_time = time.time()
    open_list, closed_set = [], set()
    heapq.heappush(open_list, (0, Puzzle(initial)))
    nodes_explored = 0

    while open_list:
        _, current = heapq.heappop(open_list)
        nodes_explored += 1

        if current.state == goal:
            return current.get_path(), nodes_explored, time.time() - start_time

        closed_set.add(tuple(map(tuple, current.state)))
        for neighbor in current.get_neighbors():
            if tuple(map(tuple, neighbor.state)) in closed_set:
                continue
            neighbor.cost = neighbor.depth + heuristic(neighbor.state, goal)
            heapq.heappush(open_list, (neighbor.cost, neighbor))

    return None, nodes_explored, time.time() - start_time

def greedy_best_first_search(initial, goal, heuristic):
    start_time = time.time()
    open_list, closed_set = [], set()
    heapq.heappush(open_list, (heuristic(initial, goal), Puzzle(initial)))
    nodes_explored = 0

    while open_list:
        _, current = heapq.heappop(open_list)
        nodes_explored += 1

        if current.state == goal:
            return current.get_path(), nodes_explored, time.time() - start_time

        closed_set.add(tuple(map(tuple, current.state)))
        for neighbor in current.get_neighbors():
            if tuple(map(tuple, neighbor.state)) in closed_set:
                continue
            heapq.heappush(open_list, (heuristic(neighbor.state, goal), neighbor))

    return None, nodes_explored, time.time() - start_time

initial_state = [[1, 2, 3], [4, 0, 5], [6, 7, 8]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

algorithms = {
    "A* (Misplaced Tiles)": lambda: a_star(initial_state, goal_state, heuristic_misplaced_tiles),
    "A* (Manhattan)": lambda: a_star(initial_state, goal_state, heuristic_manhattan),
    "GBFS (Misplaced Tiles)": lambda: greedy_best_first_search(initial_state, goal_state, heuristic_misplaced_tiles),
    "GBFS (Manhattan)": lambda: greedy_best_first_search(initial_state, goal_state, heuristic_manhattan),
}

print("Algorithm | Nodes Explored | Solution Depth | Execution Time (s)")
print("-" * 60)
for name, func in algorithms.items():
    path, nodes, exec_time = func()
    print(f"{name:<25} | {nodes:<15} | {len(path) if path else 'N/A':<14} | {exec_time:.4f}")


Algorithm | Nodes Explored | Solution Depth | Execution Time (s)
------------------------------------------------------------
A* (Misplaced Tiles)      | 318             | 15             | 0.0113
A* (Manhattan)            | 128             | 15             | 0.0042
GBFS (Misplaced Tiles)    | 104             | 19             | 0.0045
GBFS (Manhattan)          | 105             | 31             | 0.0052
